In [19]:
from src import post_process
from src.visual_tools import *
import matplotlib.pyplot as plt
from skimage import morphology, io
from skimage import img_as_ubyte as as_ubyte

def run_analysis(hour, fov, input_dir, process_args, save_to_dir=None, DEBUG=False):
    imgs = np.array([])
    img_paths = np.array([])
    for fname in np.sort(os.listdir(input_dir)):
        if fname.endswith('.png') and np.char.find(fname, 'trace') == -1:
            path = os.path.join(input_dir, fname)
            img_paths = np.append(img_paths, path)
            img = io.imread(path)
            if len(img.shape) > 2:
                img = img[:,:,0]
                
            if len(imgs) == 0:
                imgs = img
            else:
                imgs = np.dstack((imgs,img))
    
    data = post_process(imgs, save_to_dir=save_to_dir, **process_args)
    
    if DEBUG:
        trace_dir = img_paths[0].split('/predict/')[0]
        trace_path = os.path.join(trace_dir, 'trace.png')

        trace = io.imread(trace_path)
        print('plotting')
        fig, axes = plt.subplots(2, 4, figsize=(15,8))

        for ax in axes.flatten():
            ax.set_axis_off()

        sub_plot(axes[0,0], data['compiled'], title='Min (logical or) Combined Image')
        sub_plot(axes[0,1], data['double_thresh'], title='Double Thresholded Combined Image')
        sub_plot(axes[0,2], data['dilated'], title='Dilated')
        sub_plot(axes[0,3], data['closed'], title='Fill Holes')
        
        sub_plot(axes[1,0], data['skeleton'], title='Double Threshold Skeleton')
        sub_plot(axes[1,1], data['pruned_skeleton'], title='Pruned Skeleton')
        sub_plot(axes[1,3], trace, title='Hand Tracing')

        fig.tight_layout()
        plt.show()
        plt.close()

    if save_to_dir is not None:
        if not os.path.isdir(save_to_dir):
            os.mkdir(save_to_dir)
        io.imsave(os.path.join(save_dir,'1_combine.png'), data['compiled'])
        io.imsave(os.path.join(save_dir,'2_thresh.png'),  as_ubyte(data['double_thresh']))
        io.imsave(os.path.join(save_dir,'3_dilate.png'),  as_ubyte(data['dilated']))
        io.imsave(os.path.join(save_dir,'4_fill.png'),    as_ubyte(data['closed']))
        io.imsave(os.path.join(save_dir,'5_skel.png'),    as_ubyte(data['skeleton']))
        io.imsave(os.path.join(save_dir,'6_pruned_skel.png'), 255 * data['pruned_skeleton'])

    return data['pruned_skeleton']

# Doing Image Compilation

In [20]:
import os
from skimage import io
import numpy as np
from plantcv import plantcv as pcv
%load_ext autoreload
%autoreload 2

skel_dir = f'../data/test_all/results 03-24 (2 dilations)'
hours = ['1hr2741',
            '1hr2745',
            '1hr2753',
            '1hr2761',
            '1hr2789',
            '2hr2312',
            '2hr2314',
            '2hr2317',
            '2hr2331',
            '2hr2349',
            '2hr2354',
            '2hr2356',
            '2hr2364',
            '4hr2413',
            '4hr2414',
            '4hr2434',
            '4hr2436',
            '4hr2465',
            '4hr2495',
            '4hr2507',
            '10hr2127',
            '10hr2136',
            '10hr2144',
            '10hr2153',
            '10hr2155',
            '10hr2159',
            '10hr2400',
            '10hr2405',
            '10hr2429']

hours = ['10hr2400']

for entry in hours:
    print(entry)
    hour = entry.split('hr')[0] + 'HR'
    fov = entry.split('hr')[-1]
    fov_dir = f'../data/test_all/{hour}/{fov}'
    input_dir = os.path.join(fov_dir, 'predict')
    save_dir = os.path.join(fov_dir, 'results/04-08/')
        
    args = {
        'compilation': 'min',
        'n_dilations': 2,
        'liberal_thresh': 200,
        'conservative_thresh': 160,
        'invert_double_thresh': True,
        'out_dict': True
    }
    
    pruned_skel = run_analysis(hour, fov, input_dir, save_to_dir=save_dir, process_args=args)
    
    '''
    if skel_dir is not None:
        io.imsave(os.path.join(skel_dir,f'{entry}.png'), 255 * pruned_skel)
    print(os.path.join(fov_dir, 'trace.png'))
    try:
        trace = io.imread(os.path.join(fov_dir, 'trace.png'))
    except FileNotFoundError:
        trace = io.imread(os.path.join(fov_dir, 'trace.tif'))
        
    if skel_dir is not None:
        io.imsave(os.path.join(skel_dir,f'{entry}_trace.png'), trace)
    '''

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
10hr2400


In [ ]:
fig, axes = plt.subplots(len(hours), 3, figsize=(13,4 * len(hours)))
    
for ax in axes.flatten():
    ax.set_axis_off()

for ind, entry in enumerate(hours):
    print(entry)
    hour = entry.split('hr')[0] + 'HR'
    fov = entry.split('hr')[-1]
    fov_dir = f'../data/test_all/{hour}/{fov}'
    
    trace_path = os.path.join(fov_dir, 'trace.png')
    trace = io.imread(trace_path)
    
    skel_path = os.path.join(skel_dir, f'{entry}.png')
    skel = io.imread(skel_path)
    
    skel_no_fill_path = os.path.join(f'../data/test_all/results 03-21', f'{entry}.png')
    skel_no_fill = io.imread(skel_no_fill_path)
    
    sub_plot(axes[ind,0], trace, title=f'{entry}\nHand Tracing', title_color='w')
    sub_plot(axes[ind,1], 255-skel, title='Skeleton With Closing', title_color='w')
    sub_plot(axes[ind,2], 255-skel_no_fill, title='Skeleton Without Closing', title_color='w')

fig.tight_layout()
print('Saving Fig')
fig.savefig('big.png', facecolor='black', bbox_inches='tight', dpi=100)
plt.close()